In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Copy_of_wgbh.csv', index_col='DateEntered', parse_dates=True)

In [2]:
# convert index to yearly DateTime object
df.index = df.index.to_period('Y')

In [3]:
df.columns

Index(['Agency', 'ProjectName', 'ProjectAddress_1', 'Developer',
       'GeneralContractor', 'SubContractor', 'SubContractorAddress_1',
       'SubContractorAddress_2', 'Trade', 'SEX', 'MINOR', 'Race_Desc',
       'RESIDENT', 'TotalHours'],
      dtype='object')

In [4]:
# drop everything but race, residency, developer columns
df = df.drop(['Agency', 'ProjectName', 'ProjectAddress_1',
       'GeneralContractor', 'SubContractor', 'SubContractorAddress_1',
       'SubContractorAddress_2', 'Trade', 'MINOR', 'TotalHours'], axis=1)

In [5]:
df = df.drop('SEX', axis=1)

In [6]:
df = df.drop(df.index[0])
race_df = df[['Developer', 'Race_Desc']]
res_df = df[['Developer', 'RESIDENT']].dropna()

In [7]:
# create a current workforce composition report by residency per developer
summary = pd.DataFrame(columns=['Developer', 'Y', 'N', 'TOTAL_PROJECTS'])
developers = res_df['Developer'].unique()
groups = res_df.groupby(res_df.Developer)
res = ['Y', 'N']
res_df = res_df.dropna()

# compute percentage of employees for each developer by residency
for i in range(len(developers)):
    temp = groups.get_group(developers[i])
    filt1 = temp['RESIDENT'] == 'Y'
    filt2 = temp['RESIDENT'] == 'y'
    filt3 = temp['RESIDENT'] == 'N'
    filt4 = temp['RESIDENT'] == 'n'
    Y = temp[filt1]
    y = temp[filt2]
    N = temp[filt3]
    n = temp[filt4]
    num_res = Y.shape[0] + y.shape[0]
    num_non_res = N.shape[0] + n.shape[0]
    summary.loc[len(summary.index)] = [developers[i], num_res, num_non_res, num_res + num_non_res]

In [8]:
summary.columns = ['Developer', 'Resident', 'Non-resident', 'Total']
summary.to_csv('Current_Developer_Workforce_Comp_By_Residency.csv')

In [9]:
summary2 = pd.DataFrame(index=developers, columns=['2016', '2017', '2018', '2019', '2020', '2021'], data=0)

In [10]:
# compute number of Boston residents employed per year for each developer
temp_years = ['2016', '2017', '2018', '2019', '2020', '2021']

for i in range(len(developers)):
    temp = groups.get_group(developers[i])
    temp_groups = temp.groupby(temp.index)
    more_years = temp.index.unique()
    for j in range(len(more_years)):
        curr_year_df = temp_groups.get_group(more_years[j])
        residencies = curr_year_df.RESIDENT.unique()
        curr_year_groups = curr_year_df.groupby(curr_year_df.RESIDENT)
        for k in range(len(residencies)):
            curr_group = curr_year_groups.get_group(residencies[k])
            if residencies[k] == 'Y' or residencies[k] == 'y':
                summary2.loc[developers[i], temp_years[j]] += curr_group.count().RESIDENT

In [11]:
summary2

,2016,2017,2018,2019,2020,2021
CITY OF BOSTON / PFD,328,1236,855,12,0,0
THE COMMUNITY BUILDERS,96,637,1137,309,11,0
CITY OF BOSTON / PFD,48,24,41,1114,1057,0
HYM INVESTMENT GROUP,261,1385,1734,465,67,0
Delware Limited Liability Co.,116,245,0,0,0,0
...,...,...,...,...,...,...
Trusties of Boston college,7,0,0,0,0,0
WEST NAPOLI PIZZA,0,0,0,0,0,0
"PARTNERS HEALTHCARE SYSTEMS, INC.",1,0,0,0,0,0
BRIGHAM AND WOMEN'S HOSPITAL,0,0,0,0,0,0


In [12]:
summary2.to_csv('Number_of_Boston_Residents_Employed_Per_Year_By_Contractor.csv')

In [13]:
races = race_df.Race_Desc.unique()
race_df = race_df.dropna()

# create a current workforce composition report by race per developer
summary3 = pd.DataFrame(columns=['Developer', 'CAUCASIAN', 'BLACK', 'HISPANIC', 'ASIAN', 'OTHER', 'Total'])
developers = race_df['Developer'].unique()
print('Beginning Grouping')
groups = race_df.groupby(race_df.Developer)
print('Done Grouping By Developer')

# compute percentage of employees for each developer by race
for i in range(len(developers)):
    temp = groups.get_group(developers[i])
    filt1 = temp['Race_Desc'] == 'CAUCASIAN'
    filt2 = temp['Race_Desc'] == 'BLACK'
    filt3 = temp['Race_Desc'] == 'ASIAN'
    filt4 = temp['Race_Desc'] == 'HISPANIC'
    filt5 = temp['Race_Desc'] == 'OTHER'
    CAUCASIAN = temp[filt1]
    BLACK = temp[filt2]
    ASIAN = temp[filt3]
    HISPANIC = temp[filt4]
    OTHER = temp[filt5]
    num_caucasian = CAUCASIAN.shape[0]
    num_black = BLACK.shape[0]
    num_asian = ASIAN.shape[0]
    num_hispanic = HISPANIC.shape[0]
    num_other = OTHER.shape[0]
    print(i)
    summary3.loc[len(summary3.index)] = [developers[i], num_caucasian, num_black, num_hispanic, num_asian, num_other, (num_caucasian + num_black + num_hispanic + num_asian + num_other)]

Beginning Grouping
Done Grouping By Developer
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [14]:
summary3.head()

,Developer,CAUCASIAN,BLACK,HISPANIC,ASIAN,OTHER,Total
0,CITY OF BOSTON / PFD,6864,1593,1452,177,113,10199
1,THE COMMUNITY BUILDERS,2904,1795,950,273,128,6050
2,CITY OF BOSTON / PFD,5060,1635,968,67,135,7865
3,HYM INVESTMENT GROUP,6758,2162,2673,521,220,12334
4,Delware Limited Liability Co.,664,209,189,14,32,1108


In [15]:
# convert absolute values into percentages
for i in range(len(summary3.index)):
    for race in races:
        summary3.loc[i, race] = float(summary3.loc[i, race] / summary3.loc[i, 'Total'])

In [16]:
summary3.head()

,Developer,CAUCASIAN,BLACK,HISPANIC,ASIAN,OTHER,Total
0,CITY OF BOSTON / PFD,0.673007,0.156192,0.142367,0.0173546,0.0110795,10199
1,THE COMMUNITY BUILDERS,0.48,0.296694,0.157025,0.045124,0.021157,6050
2,CITY OF BOSTON / PFD,0.643357,0.207883,0.123077,0.00851875,0.0171647,7865
3,HYM INVESTMENT GROUP,0.547916,0.175288,0.216718,0.042241,0.0178369,12334
4,Delware Limited Liability Co.,0.599278,0.188628,0.170578,0.0126354,0.0288809,1108


In [17]:
summary3.to_csv('Total_Developer_Workforce_Comp_By_Race.csv')

In [18]:
# compute number of Boston residents employed per year for each developer
temp_years = ['2016', '2017', '2018', '2019', '2020', '2021']

#for i in range(len(developers)):
#    temp = groups.get_group(developers[i])
#    temp_groups = temp.groupby(temp.index)
#    more_years = temp.index.unique()
#    for j in range(len(more_years)):
#        curr_year_df = temp_groups.get_group(more_years[j])
#        residencies = curr_year_df.RESIDENT.unique()
#        curr_year_groups = curr_year_df.groupby(curr_year_df.RESIDENT)
#        for k in range(len(residencies)):
#            curr_group = curr_year_groups.get_group(residencies[k])
#            if residencies[k] == 'Y' or residencies[k] == 'y':
#                summary2.loc[developers[i], temp_years[j]] += curr_group.count().RESIDENT
race_df

,Developer,Race_Desc
DateEntered,,
2021,CITY OF BOSTON / PFD,CAUCASIAN
2021,CITY OF BOSTON / PFD,CAUCASIAN
2021,CITY OF BOSTON / PFD,CAUCASIAN
2021,CITY OF BOSTON / PFD,CAUCASIAN
2021,CITY OF BOSTON / PFD,CAUCASIAN
...,...,...
2016,RTH RIVERWAY LLCI,BLACK
2016,RTH RIVERWAY LLCI,HISPANIC
2016,RTH RIVERWAY LLCI,CAUCASIAN


In [19]:
# segment data by year, creating as many CSVs as years, each showing that year's data for each developer
year_groups = race_df.groupby(race_df.index)
temp_years = race_df.index.unique()
y = ['2016', '2017', '2018', '2019', '2020', '2021']

for i in range(len(temp_years)):
    temp = year_groups.get_group(temp_years[i])
    curr_year_devs = temp.Developer.unique()
    curr_year_summary = pd.DataFrame(columns=['Developer', 'CAUCASIAN', 'BLACK', 'HISPANIC', 'ASIAN', 'OTHER', 'Total'])
    for j in range(len(curr_year_devs)):
        temp_group = groups.get_group(curr_year_devs[j])
        filt1 = temp_group['Race_Desc'] == 'CAUCASIAN'
        filt2 = temp_group['Race_Desc'] == 'BLACK'
        filt3 = temp_group['Race_Desc'] == 'ASIAN'
        filt4 = temp_group['Race_Desc'] == 'HISPANIC'
        filt5 = temp_group['Race_Desc'] == 'OTHER'
        CAUCASIAN = temp_group[filt1]
        BLACK = temp_group[filt2]
        ASIAN = temp_group[filt3]
        HISPANIC = temp_group[filt4]
        OTHER = temp_group[filt5]
        num_caucasian = CAUCASIAN.shape[0]
        num_black = BLACK.shape[0]
        num_asian = ASIAN.shape[0]
        num_hispanic = HISPANIC.shape[0]
        num_other = OTHER.shape[0]
        print(j)
        curr_year_summary.loc[len(curr_year_summary.index)] = [curr_year_devs[j], num_caucasian, num_black, num_hispanic, num_asian, num_other, (num_caucasian + num_black + num_hispanic + num_asian + num_other)]
    # convert absolute values into percentages
    for k in range(len(curr_year_summary.index)):
        for race in races:
            curr_year_summary.loc[k, race] = float(curr_year_summary.loc[k, race] / curr_year_summary.loc[k, 'Total'])
    curr_year_summary.to_csv(y[i] + '_Developer_Workforce_Composition_By_Race.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
